In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

# ============================================================================
# SIMPLE PSO CLUSTERING
# ============================================================================

class PSOClustering:
    """Particle Swarm Optimization for Clustering - Simplified"""
    
    def __init__(self, num_clusters=4, num_particles=30, num_iterations=100):
        self.num_clusters = num_clusters
        self.num_particles = num_particles
        self.num_iterations = num_iterations
        self.centroids = None
        self.convergence = []
        
    def _calculate_fitness(self, centroids, data):
        """Calculate within-cluster sum of squares"""
        distances = np.linalg.norm(data[:, None] - centroids[None, :], axis=2)
        labels = np.argmin(distances, axis=1)
        
        # Sum of squared distances
        fitness = sum(np.sum((data[labels == k] - centroids[k])**2) 
                     for k in range(self.num_clusters) 
                     if np.sum(labels == k) > 0)
        
        return fitness
    
    def fit(self, data):
        """Run PSO optimization"""
        n_features = data.shape[1]
        
        # Initialize particles
        positions = np.array([data[np.random.choice(len(data), self.num_clusters)] 
                             for _ in range(self.num_particles)])
        velocities = np.zeros_like(positions)
        
        # Personal and global bests
        personal_best_positions = positions.copy()
        personal_best_scores = np.array([self._calculate_fitness(p, data) 
                                         for p in positions])
        
        global_best_idx = np.argmin(personal_best_scores)
        global_best_position = personal_best_positions[global_best_idx].copy()
        global_best_score = personal_best_scores[global_best_idx]
        
        # PSO parameters
        w = 0.7   # inertia
        c1 = 1.5  # cognitive
        c2 = 1.5  # social
        
        print(f"Running PSO with {self.num_particles} particles, {self.num_iterations} iterations...")
        
        # Main PSO loop
        for iteration in range(self.num_iterations):
            for i in range(self.num_particles):
                # Update velocity
                r1, r2 = np.random.rand(), np.random.rand()
                velocities[i] = (w * velocities[i] + 
                               c1 * r1 * (personal_best_positions[i] - positions[i]) +
                               c2 * r2 * (global_best_position - positions[i]))
                
                # Update position
                positions[i] += velocities[i]
                
                # Evaluate fitness
                fitness = self._calculate_fitness(positions[i], data)
                
                # Update personal best
                if fitness < personal_best_scores[i]:
                    personal_best_scores[i] = fitness
                    personal_best_positions[i] = positions[i].copy()
                    
                    # Update global best
                    if fitness < global_best_score:
                        global_best_score = fitness
                        global_best_position = positions[i].copy()
            
            self.convergence.append(global_best_score)
            
            if (iteration + 1) % 20 == 0:
                print(f"  Iteration {iteration+1}/{self.num_iterations} - Best Score: {global_best_score:.2f}")
        
        self.centroids = global_best_position
        print(f"Optimization complete! Final score: {global_best_score:.2f}")
        return self
    
    def predict(self, data):
        """Assign cluster labels"""
        distances = np.linalg.norm(data[:, None] - self.centroids[None, :], axis=2)
        return np.argmin(distances, axis=1)


# ============================================================================
# LOAD AND PREPROCESS DATA
# ============================================================================

print("Loading data...")
df = pd.read_csv("/content/SCOA_A7.csv")

print(f"Dataset: {df.shape[0]} rows, {df.shape[1]} columns")
print("\nFirst few rows:")
print(df.head())

# Encode Gender as numeric feature
df['Gender_Encoded'] = df['Gender'].map({'Male': 0, 'Female': 1})

# Select features for clustering (including Gender)
features = ['Gender_Encoded', 'Age', 'Annual Income (k$)', 'Spending Score (1-100)']
feature_names = ['Gender', 'Age', 'Income', 'Spending Score']
data_raw = df[features].values

# Standardize features
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_raw)

print(f"\nUsing features: {feature_names}")

# ============================================================================
# RUN PSO CLUSTERING
# ============================================================================

num_clusters = 5
pso = PSOClustering(num_clusters=num_clusters, num_particles=500, num_iterations=100)
pso.fit(data_scaled)

# Get cluster labels
labels = pso.predict(data_scaled)

# Transform centroids back to original scale
centroids_original = scaler.inverse_transform(pso.centroids)

# ============================================================================
# RESULTS
# ============================================================================

print("\n" + "="*70)
print("CLUSTERING RESULTS")
print("="*70)

# Cluster distribution
print("\nCluster sizes:")
for i in range(num_clusters):
    count = np.sum(labels == i)
    print(f"  Cluster {i}: {count} customers ({count/len(labels)*100:.1f}%)")

# Silhouette score
if len(np.unique(labels)) > 1:
    sil_score = silhouette_score(data_scaled, labels)
    print(f"\nSilhouette Score: {sil_score:.3f} (higher is better)")

# Cluster profiles
print("\nCluster Centroids:")
print("-" * 70)
print(f"{'Cluster':<10} {'Gender':<12} {'Age':<10} {'Income($k)':<15} {'Spending':<10}")
print("-" * 70)
for i in range(num_clusters):
    gender_val = "Male" if centroids_original[i,0] < 0.5 else "Female"
    print(f"{i:<10} {gender_val:<12} {centroids_original[i,1]:<10.1f} "
          f"{centroids_original[i,2]:<15.1f} {centroids_original[i,3]:<10.1f}")

# Add cluster labels to dataframe
df['Cluster'] = labels

print("\nCluster Characteristics:")
print("-" * 70)
for i in range(num_clusters):
    cluster_data = df[df['Cluster'] == i]
    print(f"\nCluster {i}:")
    print(f"  Size: {len(cluster_data)} customers")
    print(f"  Average Age: {cluster_data['Age'].mean():.1f}")
    print(f"  Average Income: ${cluster_data['Annual Income (k$)'].mean():.1f}k")
    print(f"  Average Spending Score: {cluster_data['Spending Score (1-100)'].mean():.1f}")
    gender_counts = cluster_data['Gender'].value_counts()
    male_pct = (gender_counts.get('Male', 0) / len(cluster_data)) * 100
    female_pct = (gender_counts.get('Female', 0) / len(cluster_data)) * 100
    print(f"  Gender: {male_pct:.0f}% Male, {female_pct:.0f}% Female")

# ============================================================================
# VISUALIZATIONS
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Main clustering result (Income vs Spending, colored by cluster)
ax1 = axes[0]

# Separate Male and Female for different markers
male_mask = df['Gender'] == 'Male'
female_mask = df['Gender'] == 'Female'

# Plot males
scatter_m = ax1.scatter(df[male_mask]['Annual Income (k$)'], 
                        df[male_mask]['Spending Score (1-100)'], 
                        c=labels[male_mask], cmap='viridis', 
                        s=60, alpha=0.6, marker='s',
                        edgecolors='black', linewidth=0.5, label='Male')

# Plot females
scatter_f = ax1.scatter(df[female_mask]['Annual Income (k$)'], 
                        df[female_mask]['Spending Score (1-100)'], 
                        c=labels[female_mask], cmap='viridis', 
                        s=60, alpha=0.6, marker='o',
                        edgecolors='black', linewidth=0.5, label='Female')

# Plot centroids
ax1.scatter(centroids_original[:, 2], centroids_original[:, 3], 
            c='red', marker='X', s=400, edgecolors='black', 
            linewidth=2, label='Centroids', zorder=5)

ax1.set_xlabel('Annual Income (k$)', fontsize=12)
ax1.set_ylabel('Spending Score (1-100)', fontsize=12)
ax1.set_title('Customer Segmentation (PSO with Gender)\n■ = Male, ● = Female', 
              fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Add cluster labels to plot
for i in range(num_clusters):
    gender_str = "M" if centroids_original[i, 0] < 0.5 else "F"
    ax1.annotate(f'C{i}({gender_str})', 
                xy=(centroids_original[i, 2], centroids_original[i, 3]),
                xytext=(10, 10), textcoords='offset points',
                fontsize=10, fontweight='bold', color='darkred')

# Plot 2: Convergence curve
ax2 = axes[1]
ax2.plot(pso.convergence, linewidth=2.5, color='darkblue')
ax2.set_xlabel('Iteration', fontsize=12)
ax2.set_ylabel('Fitness (WCSS)', fontsize=12)
ax2.set_title('PSO Convergence', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('pso_clustering_with_gender.png', dpi=300, bbox_inches='tight')
print("\n" + "="*70)
print("Visualization saved as 'pso_clustering_with_gender.png'")
print("="*70)
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

# ============================================================================
# SIMPLE PSO CLUSTERING
# ============================================================================

class PSOClustering:
    """Particle Swarm Optimization for Clustering - Simplified"""
    
    def __init__(self, num_clusters=4, num_particles=30, num_iterations=100):
        self.num_clusters = num_clusters
        self.num_particles = num_particles
        self.num_iterations = num_iterations
        self.centroids = None
        self.convergence = []
        
    def _calculate_fitness(self, centroids, data):
        """Calculate within-cluster sum of squares"""
        distances = np.linalg.norm(data[:, None] - centroids[None, :], axis=2)
        labels = np.argmin(distances, axis=1)
        
        # Sum of squared distances
        fitness = sum(np.sum((data[labels == k] - centroids[k])**2) 
                     for k in range(self.num_clusters) 
                     if np.sum(labels == k) > 0)
        
        return fitness
    
    def fit(self, data):
        """Run PSO optimization"""
        n_features = data.shape[1]
        
        # Initialize particles
        positions = np.array([data[np.random.choice(len(data), self.num_clusters)] 
                             for _ in range(self.num_particles)])
        velocities = np.zeros_like(positions)
        
        # Personal and global bests
        personal_best_positions = positions.copy()
        personal_best_scores = np.array([self._calculate_fitness(p, data) 
                                         for p in positions])
        
        global_best_idx = np.argmin(personal_best_scores)
        global_best_position = personal_best_positions[global_best_idx].copy()
        global_best_score = personal_best_scores[global_best_idx]
        
        # PSO parameters
        w = 0.7   # inertia
        c1 = 1.5  # cognitive
        c2 = 1.5  # social
        
        print(f"Running PSO with {self.num_particles} particles, {self.num_iterations} iterations...")
        
        # Main PSO loop
        for iteration in range(self.num_iterations):
            for i in range(self.num_particles):
                # Update velocity
                r1, r2 = np.random.rand(), np.random.rand()
                velocities[i] = (w * velocities[i] + 
                               c1 * r1 * (personal_best_positions[i] - positions[i]) +
                               c2 * r2 * (global_best_position - positions[i]))
                
                # Update position
                positions[i] += velocities[i]
                
                # Evaluate fitness
                fitness = self._calculate_fitness(positions[i], data)
                
                # Update personal best
                if fitness < personal_best_scores[i]:
                    personal_best_scores[i] = fitness
                    personal_best_positions[i] = positions[i].copy()
                    
                    # Update global best
                    if fitness < global_best_score:
                        global_best_score = fitness
                        global_best_position = positions[i].copy()
            
            self.convergence.append(global_best_score)
            
            if (iteration + 1) % 20 == 0:
                print(f"  Iteration {iteration+1}/{self.num_iterations} - Best Score: {global_best_score:.2f}")
        
        self.centroids = global_best_position
        print(f"Optimization complete! Final score: {global_best_score:.2f}")
        return self
    
    def predict(self, data):
        """Assign cluster labels"""
        distances = np.linalg.norm(data[:, None] - self.centroids[None, :], axis=2)
        return np.argmin(distances, axis=1)


# ============================================================================
# LOAD AND PREPROCESS DATA
# ============================================================================

print("Loading data...")
df = pd.read_csv("/content/SCOA_A7.csv")

print(f"Dataset: {df.shape[0]} rows, {df.shape[1]} columns")
print("\nFirst few rows:")
print(df.head())

# Select features for clustering
features = ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']
data_raw = df[features].values

# Standardize features
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_raw)

print(f"\nUsing features: {features}")

# ============================================================================
# RUN PSO CLUSTERING
# ============================================================================

num_clusters = 6
pso = PSOClustering(num_clusters=num_clusters, num_particles=1000, num_iterations=150)
pso.fit(data_scaled)

# Get cluster labels
labels = pso.predict(data_scaled)

# Transform centroids back to original scale
centroids_original = scaler.inverse_transform(pso.centroids)

# ============================================================================
# RESULTS
# ============================================================================

print("\n" + "="*60)
print("CLUSTERING RESULTS")
print("="*60)

# Cluster distribution
print("\nCluster sizes:")
for i in range(num_clusters):
    count = np.sum(labels == i)
    print(f"  Cluster {i}: {count} customers ({count/len(labels)*100:.1f}%)")

# Silhouette score
if len(np.unique(labels)) > 1:
    sil_score = silhouette_score(data_scaled, labels)
    print(f"\nSilhouette Score: {sil_score:.3f} (higher is better)")

# Cluster profiles
print("\nCluster Centroids:")
print("-" * 60)
print(f"{'Cluster':<10} {'Age':<10} {'Income($k)':<15} {'Spending':<10}")
print("-" * 60)
for i in range(num_clusters):
    print(f"{i:<10} {centroids_original[i,0]:<10.1f} "
          f"{centroids_original[i,1]:<15.1f} {centroids_original[i,2]:<10.1f}")

# Add cluster labels to dataframe
df['Cluster'] = labels

print("\nCluster Characteristics:")
print("-" * 60)
for i in range(num_clusters):
    cluster_data = df[df['Cluster'] == i]
    print(f"\nCluster {i}:")
    print(f"  Average Age: {cluster_data['Age'].mean():.1f}")
    print(f"  Average Income: ${cluster_data['Annual Income (k$)'].mean():.1f}k")
    print(f"  Average Spending Score: {cluster_data['Spending Score (1-100)'].mean():.1f}")

# ============================================================================
# VISUALIZATIONS
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Main clustering result (Income vs Spending)
ax1 = axes[0]
scatter = ax1.scatter(data_raw[:, 1], data_raw[:, 2], 
                      c=labels, cmap='viridis', s=60, 
                      alpha=0.6, edgecolors='black', linewidth=0.5)
ax1.scatter(centroids_original[:, 1], centroids_original[:, 2], 
            c='red', marker='X', s=400, edgecolors='black', 
            linewidth=2, label='Centroids', zorder=5)
ax1.set_xlabel('Annual Income (k$)', fontsize=12)
ax1.set_ylabel('Spending Score (1-100)', fontsize=12)
ax1.set_title('Customer Segmentation (PSO Clustering)', fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Add cluster labels to plot
for i in range(num_clusters):
    ax1.annotate(f'C{i}', 
                xy=(centroids_original[i, 1], centroids_original[i, 2]),
                xytext=(10, 10), textcoords='offset points',
                fontsize=11, fontweight='bold', color='darkred')

# Plot 2: Convergence curve
ax2 = axes[1]
ax2.plot(pso.convergence, linewidth=2.5, color='darkblue')
ax2.set_xlabel('Iteration', fontsize=12)
ax2.set_ylabel('Fitness (WCSS)', fontsize=12)
ax2.set_title('PSO Convergence', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('pso_clustering_results.png', dpi=300, bbox_inches='tight')
print("\n" + "="*60)
print("Visualization saved as 'pso_clustering_results.png'")
print("="*60)
plt.show()

# PSO Clustering — Detailed Explanation

> This document explains the provided Particle Swarm Optimization (PSO) clustering script in depth. It covers the algorithmic concepts, code walkthrough, interpretation of outputs, suggested experiments and improvements, and practical tips for reproducible and meaningful clustering.

---

## Table of contents

1. High-level summary
2. Background: Particle Swarm Optimization (PSO)

   * Origins and intuition
   * PSO components and equations
   * PSO for clustering: representation and fitness
3. Data preprocessing and scaling: why it matters
4. Code walkthrough (module-by-module)

   * Class `PSOClustering`
   * Fitness calculation (WCSS)
   * Particle initialization and velocities
   * Personal/global bests and PSO loop
   * Predict method
   * Script outside the class: data loading, scaling, fitting, and visualization
5. Understanding the results

   * WCSS and convergence curve
   * Silhouette score and cluster validity
   * Interpreting centroids and cluster characteristics
6. Practical considerations, hyperparameters and recommended ranges

   * num_particles
   * num_iterations
   * num_clusters
   * PSO constants (w, c1, c2)
   * Initialization and multiple runs
7. Common pitfalls and debugging tips
8. Possible improvements and extensions

   * Elitism, topology, and hybridization
   * Different fitness functions and regularization
   * Handling empty clusters
   * Parallel evaluation
9. Visualization and analysis to report
10. Reproducibility and computational cost
11. Quick checklist

---

# 1. High-level summary

The script implements a **simplified Particle Swarm Optimization (PSO)** approach to clustering. Instead of traditional k-means, PSO treats each candidate solution (particle) as a set of cluster centroids. Particles move in the data space to minimize the within-cluster sum of squares (WCSS). The final centroids are returned and used to label the dataset. The script also computes cluster sizes, average feature values per cluster, and the silhouette score, and visualizes clusters and PSO convergence.

# 2. Background: Particle Swarm Optimization (PSO)

## Origins and intuition

PSO is an optimization technique introduced by Kennedy and Eberhart in 1995, inspired by flocking birds or schooling fish. Each particle represents a candidate solution and has a position and velocity in the search space. Particles adjust their velocity based on their own experience (personal best) and the swarm’s experience (global best), balancing exploration and exploitation.

## PSO components and equations

For particle *i* at iteration *t*:

* Position: (x_i^t)
* Velocity: (v_i^t)
* Personal best position: (p_i^t)
* Global best position: (g^t)

Velocity update (simplified):

[ v_i^{t+1} = w v_i^t + c_1 r_1 (p_i^t - x_i^t) + c_2 r_2 (g^t - x_i^t) ]

Position update:

[ x_i^{t+1} = x_i^t + v_i^{t+1} ]

Where:

* (w) is inertia (controls momentum),
* (c_1) cognitive coefficient (attraction to personal best),
* (c_2) social coefficient (attraction to global best),
* (r_1, r_2) are uniform random numbers in ([0,1]).

These terms let particles exploit known good regions while still exploring.

## PSO for clustering: representation and fitness

* **Representation**: Each particle encodes `num_clusters` centroids. For data with `d` features, particle position has shape `(num_clusters, d)`. The PSO operates in this high-dimensional continuous space.
* **Fitness**: The script uses **Within-Cluster Sum of Squares (WCSS)**: sum of squared Euclidean distances between points and their assigned centroid. PSO seeks to minimize this—just like k-means.

# 3. Data preprocessing and scaling: why it matters

Features are standardized with `StandardScaler`. This is crucial because PSO moves centroids in Euclidean space. If features are on different scales (age in years vs income in thousands vs spending score 1–100), centroids will be biased towards larger-scale dimensions. Standardization ensures each feature contributes equally to distance calculations.

# 4. Code walkthrough (module-by-module)

> The class-based design isolates PSO logic. Below are important internals.

### `PSOClustering.__init__`

Sets `num_clusters`, `num_particles`, `num_iterations`. Also prepares placeholders for `centroids` and `convergence` tracking.

### `_calculate_fitness(centroids, data)`

* Calculates pairwise distances between data points and centroids.
* Assigns labels by argmin distances.
* Computes WCSS for non-empty clusters only.
* Returns scalar fitness (smaller is better).

Note: the function ignores empty clusters (clusters with zero assigned points) by skipping them in summation. This prevents NaN but does not penalize empty centroids explicitly.

### `fit(data)`

* **Initialization**:

  * `positions`: randomly samples `num_clusters` points from the data for each particle. This produces an initial centroid set close to actual data distribution.
  * `velocities`: initialized to zero (no initial momentum).
  * `personal_best_positions/scores`: set to initial state.
  * `global_best_position/score`: best of personal bests.

* **PSO parameters**: inertia `w=0.7`, cognitive `c1=1.5`, social `c2=1.5` — reasonable defaults balancing exploration and exploitation.

* **Main loop**:

  * For each particle, velocity updated with stochastic coefficients `r1` and `r2`.
  * Position updated by adding velocity.
  * Fitness evaluated for new position.
  * If fitness improved, update personal best; if also better than global best, update global best.
  * After all particles updated, append global best score to convergence list.

* **Logging**: prints best score every 20 iterations.

* **Return**: sets `self.centroids` to `global_best_position` (centroids in scaled space).

### `predict(data)`

Assigns each data point to the nearest centroid using Euclidean distance (in whichever feature space `data` is passed — typically scaled).

### Script outside class

* Loads dataset `SCOA_A7.csv` and prints head.
* Selects three features and scales them.
* Instantiates `PSOClustering` with `num_clusters=6`, `num_particles=1000`, `num_iterations=150`.
* Fits PSO, predicts labels, transforms centroids back to original scale, prints cluster stats, computes silhouette score, visualizes cluster scatter and convergence curve, saves figure.

# 5. Understanding the results

### WCSS and convergence curve

* `pso.convergence` stores the best WCSS found per iteration. A decreasing curve indicates optimization progress. Plateaus suggest convergence.

* The absolute WCSS value is scale-dependent; use relative improvements and plots to assess optimization.

### Silhouette score

* Silhouette ranges from -1 to +1; higher is better. Values > 0.5 indicate well-separated clusters; 0.25–0.5 are reasonable; close to 0 suggest overlapping clusters.

* Because silhouette uses distances normalized per sample, it complements WCSS by measuring separation and cohesion.

### Centroids and cluster characteristics

* Centroids printed in original scale show interpretable cluster profiles (e.g., young-high-spenders vs older-low-income).

* Cluster sizes reveal whether any cluster is empty or tiny—empty clusters indicate either too many clusters or bad initialization.

# 6. Practical considerations, hyperparameters and recommended ranges

* `num_particles`: 30–200 for most datasets. The script uses 1000 which can be highly redundant and computationally expensive. Larger swarms improve exploration but cost more evaluations. Start small (50) and scale if needed.

* `num_iterations`: 50–300 depending on problem size. 150 is reasonable but check convergence.

* `num_clusters`: use domain knowledge, elbow method, silhouette analysis, or run multiple `k` values.

* `w` (inertia): 0.4–0.9. Lower encourages exploitation; higher keeps momentum.

* `c1`, `c2`: common to set both to 1.5–2.0. If social term dominates, swarm converges quickly to global best; if cognitive dominates, particles explore around their personal best.

* **Multiple runs**: because of randomness, run PSO multiple times and pick the best solution or aggregate statistics.

# 7. Common pitfalls and debugging tips

* **Very large `num_particles`** → massive runtime: reduce or parallelize.
* **Empty clusters**: initialization and movement can produce centroids with no assigned points—consider repopulating empty centroids to random data points.
* **Diverging velocities/positions**: PSO in unconstrained space can move centroids far away. Consider velocity clamping or position bounds (e.g., min/max feature values).
* **Scaling errors**: forgetting to inverse-transform centroids before interpretation will mislead analysis.

# 8. Possible improvements and extensions

* **Elitism / archive**: keep the best-so-far explicitly across runs to avoid losing solutions.
* **Neighborhood topologies**: use local neighborhoods (ring, lattice) for social influence instead of global best to maintain diversity.
* **Hybrid PSO-kmeans**: after PSO converges, run k-means initialized at the best centroids to fine-tune.
* **Coping with empty clusters**: when cluster has zero points, re-initialize its centroid using a randomly sampled point.
* **Different fitnesses**: combine WCSS with a penalty for imbalance (too-small clusters), or use silhouette-based fitness.
* **Parallel fitness evaluation**: use `joblib` or multiprocessing to compute particle fitnesses concurrently.

# 9. Visualization and analysis to report

* Plot convergence curve (already implemented).
* Plot cluster sizes and feature distributions per cluster (boxplots or violin plots).
* Use PCA or t-SNE to visualize clusters in 2D if features > 2.
* Report silhouette and other cluster validity indices (Calinski-Harabasz, Davies-Bouldin).

# 10. Reproducibility and computational cost

* Set `np.random.seed(...)` and `random.seed(...)` near the start for reproducible experiments.
* With `num_particles=1000` and `num_iterations=150`, and 5–10k datapoints, this script can be computationally heavy because each fitness eval loops over the dataset. Consider reducing particle count or using vectorized/parallel fitness evaluation.

# 11. Quick checklist

* [ ] Standardize features (done)
* [ ] Run PSO multiple times and average results
* [ ] Try smaller `num_particles` (e.g., 50–200)
* [ ] Add handling for empty clusters
* [ ] Consider velocity clamping and bounds
* [ ] Evaluate final centroids with an independent holdout or cross-validated clustering metrics

---

*End of file.*
